<a href="https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Blog_Post_Writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Blog Post Writer
This colab showcases how Pi can help build a blog post writer in the tone and style of an existing blog. For this demonstration we are using machinelearningmastery.com blogs as inspiration. A condensed version of these blogs were scraped and loaded into Hugging Face for this colab at: [withpi/machinelearningmastery_com_blogs_condensed](https://huggingface.co/datasets/withpi/machinelearningmastery_com_blogs_condensed)

Here is the overall flow of the colab:

1.   We will first create a Pi scoring system for the blog post writer
2.   Then we will evaluate a prompted model against this contract
3.   We will then fine tune a model, by picking high quality blogs from the hugging face dataset above using Pi

4.   Finally we will use Pi scoring system to evaluate the fine tuned model against the prompted model to see if we observe any improvement




# Install packages and utility functions
Here we are installing the Pi SDK, and we're also importing a few additonal things to help out this use case including a dataset utility as well as functions to help us more legibly print scores and Side by Side comparisons

In [1]:
# @title Install necessary packages
%%capture
%pip install withpi withpi-utils
%pip install datasets
%pip install litellm
%pip install httpx jinja2 tqdm

In [2]:
# @title Intitialize PiClient
import os
from google.colab import files, userdata
from withpi import PiClient

# Load the notebook secret into the environment so the Pi Client can access it.
os.environ["WITHPI_API_KEY"] = userdata.get('WITHPI_API_KEY')


client = PiClient()

# Define your scoring system
This is where we define the set of criteria that we want to use to assess and guide the quality of our blog post generation. We'll focus on a couple categories of quality. We will use the "Contract" functions in the Pi SDK for this.


*   **Content structure:** is the post easy to digest and engaging, and does it guide the user to additional resources?
*  **Technical communication:** does the post use effective code examples and communicate any potential implementation pitfalls or mistakes



In [3]:
# @title Initialize the Pi scoring system from a JSON description
from withpi.types import Scorer

blog_writer_scorer_json = """
{
  "description": "A streamlined rubric for evaluating technical blog post quality",
  "name": "Technical Blog Post Quality Assessment",
  "dimensions": [
    {
      "description": "Evaluates the content structure of the blog post",
      "label": "Content Structure",
      "sub_dimensions": [
        {
          "description": "Are there visual breaks (images, code snippets, bullet points) to break up the text?",
          "label": "Visual breaks",
          "scoring_type": "PI_SCORER",
          "weight": 1
        },
        {
          "description": "Does the blog post address the reader in second person (you, your etc.)?",
          "label": "Second person",
          "scoring_type": "PI_SCORER",
          "weight": 1
        },
        {
          "description": "Does the post include links to additional resources or references?",
          "label": "Additional resources",
          "scoring_type": "PI_SCORER",
          "weight": 1
        },
        {
          "description": "Are there consistent section headings throughout the post?",
          "label": "Section headings",
          "scoring_type": "PI_SCORER",
          "weight": 1
        }
      ],
      "weight": 1
    },
    {
      "description": "Evaluates the technical communication of the blog post",
      "label": "Technical Communication",
      "sub_dimensions": [
        {
          "description": "Are code examples included where relevant?",
          "label": "Code inclusion",
          "scoring_type": "PI_SCORER",
          "weight": 1
        },
        {
          "description": "Does the post explain the code snippets when they are included?",
          "label": "Code explanation",
          "scoring_type": "PI_SCORER",
          "weight": 1
        },
        {
          "description": "Does the post call out potential pitfalls or common mistakes?",
          "label": "Pitfalls",
          "scoring_type": "PI_SCORER",
          "weight": 1
        }
      ],
      "weight": 1
    }
  ]
}
"""
blog_writer_scorer = Scorer.model_validate_json(blog_writer_scorer_json)

# Try Generating Blog Posts
Once we have a scoring system, let's assess how well prompting a model works for generating blog posts by


1. Define a system prompt
2. Prompt a Llama model to generate responses for a set of user prompts
3. Use our scoring system to compare the generated outputs against a set of actual [blog posts from MachineLearningMastery.com that we'd previously scraped and stored in HuggingFace](https://huggingface.co/datasets/withpi/mlmastery_com_blogs_condensed_merged)
4. Manually inspect some of the differences in the above



In [4]:
# @title Define a system prompt for a blog post generator
system_prompt_for_blog_writer = """
You are a specialized blog post writer. Given a topic, write a technical blog post. Here are specific instructions:
- Make sure that the blog is approximately under 500 words
- The blog should be technical in nature with clear instructions
"""

In [5]:
# @title Define a blog post generator
import litellm
import asyncio

async def generate_blogs(topics, system_prompt, model_id, api_base, api_key, concurrency_limit=5):
    """Generate blogs for all topics with TaskGroup and rate limiting"""
    # Create a semaphore to limit concurrent API calls
    semaphore = asyncio.Semaphore(concurrency_limit)

    async def generate_single_blog(topic, index):
        """Process a single blog generation with rate limiting"""
        async with semaphore:
            try:
                response = await litellm.acompletion(
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": topic},
                    ],
                    model=model_id,
                    api_base=api_base,
                    api_key=api_key,
                    temperature=0.2,
                )
                generated_blog = response.choices[0].message.content
                print(f"Generated a blog for topic# {index}: {topic}")
                return generated_blog
            except Exception as e:
                print(f"Error generating blog for topic# {index}: {e}")
                return f"Error: {str(e)}"

    generated_blogs = []

    # Using TaskGroup for cleaner task management
    async with asyncio.TaskGroup() as tg:
        tasks = [
            tg.create_task(generate_single_blog(topic, i + 1))
            for i, topic in enumerate(topics)
        ]

    # Collect results in the same order as topics
    for task in tasks:
        generated_blogs.append(task.result())

    print("Done generating blogs!!")
    return generated_blogs

In [6]:
# @title Generate blogs using an untrained model for evaluation
from datasets import load_dataset
from google.colab import userdata

ds = load_dataset("withpi/mlmastery_com_blogs_condensed_merged", split="test")
topics = ds["topic"]
actual_blogs = ds["blog"]

# Generate the blogs using an untrained llama 8B
loop = asyncio.get_running_loop()
generated_blogs = await loop.create_task(
    generate_blogs(
        topics,
        model_id="fireworks_ai/llama-v3p1-8b-instruct",
        api_key=userdata.get("FIREWORKS_API_KEY"),
        api_base = None,
        system_prompt=system_prompt_for_blog_writer
    )
)

README.md:   0%|          | 0.00/403 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/322k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/305 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/78 [00:00<?, ? examples/s]

Generated a blog for topic# 1: InterviewAce: 365 Data Science's Free AI-Powered Tool for Data Science Interview Preparation
Generated a blog for topic# 3: A comprehensive guide to Hugging Face's Model Hub and Community platform, including repository management, search functionality, API integration, and community resources for machine learning practitioners.
Generated a blog for topic# 4: Evaluating RAG Systems: An Overview of RAGAs and Other Frameworks for Measuring Retrieval Augmented Generation Performance
Generated a blog for topic# 5: 7-Day Mini-Course on Practical Data Science: From Linear Regression to Random Forests with Real-World Country Data Analysis
Generated a blog for topic# 2: Topic: 7 Beginner-Friendly Machine Learning Projects for Hands-On Experience - From Titanic Survival Prediction to Face Detection
Generated a blog for topic# 6: Best practices for efficient machine learning model deployment, including optimization, containerization, CI/CD implementation, performanc

In [7]:
# @title Compare the generated blogs against actual blogs using the Pi scoring system
from tqdm import tqdm
import pandas as pd

scores = []
actual_scores = []
generated_scores = []
for topic, actual_blog, generated_blog in tqdm(zip(topics, actual_blogs, generated_blogs)):
  actual_score = client.scoring_system.score(
      llm_input=topic,
      llm_output=actual_blog,
      scorer=blog_writer_scorer)
  generated_score = client.scoring_system.score(
      llm_input=topic,
      llm_output=generated_blog,
      scorer=blog_writer_scorer)
  actual_scores.append(actual_score)
  generated_scores.append(generated_score)
  score = {'topic': topic, 'actual': actual_score.total_score, 'generated': generated_score.total_score}
  scores.append(score)

df = pd.DataFrame(scores)
df

78it [00:44,  1.77it/s]


,topic,actual,generated
0,InterviewAce: 365 Data Science's Free AI-Power...,0.447750,0.222490
1,Topic: 7 Beginner-Friendly Machine Learning Pr...,0.468502,0.397787
2,A comprehensive guide to Hugging Face's Model ...,0.584229,0.297150
3,Evaluating RAG Systems: An Overview of RAGAs a...,0.510825,0.296236
4,7-Day Mini-Course on Practical Data Science: F...,0.775716,0.462908
...,...,...,...
73,Packaging and Deploying Python Projects: From ...,0.792928,0.524812
74,Creating and Customizing Dataset Classes in Py...,0.598328,0.321944
75,Implementing Dropout Regularization in PyTorch...,0.805664,0.473637
76,Building and Training a Single Layer Neural Ne...,0.726562,0.446015


In [8]:
# @title Manually inspect actual and generated blogs with scores
from withpi_utils.colab import pretty_print_responses

def pretty_print_blog(i):
  pretty_print_responses(
      response1 = actual_blogs[i].strip("\"").replace("\\n", "\n"),
      response2 = generated_blogs[i].strip("\"").replace("\\n", "\n"),
      header="##### Topic: \n" + topics[i].strip("\"").replace("\\n", "\n"),
      left_label="Base (actual)",
      right_label="Test (generated)",
      scores_left=actual_scores[i],
      scores_right=generated_scores[i])

for i in range(10):
  pretty_print_blog(i)
  print("\n\n")

Content Structure,,0.849
,Visual breaks,1.0
,Second person,0.396
,Additional resources,1.0
,Section headings,1.0
Technical Communication,,0.046
,Code inclusion,0.0
,Code explanation,0.0
,Pitfalls,0.139
Total score,,0.448
Content Structure,,0.345


Content Structure,,0.937
,Visual breaks,0.746
,Second person,1.0
,Additional resources,1.0
,Section headings,1.0
Technical Communication,,0.0
,Code inclusion,0.001
,Code explanation,0.0
,Pitfalls,0.0
Total score,,0.469
Content Structure,,0.52


Content Structure,,0.914
,Visual breaks,0.969
,Second person,0.77
,Additional resources,1.0
,Section headings,0.918
Technical Communication,,0.254
,Code inclusion,0.68
,Code explanation,0.076
,Pitfalls,0.008
Total score,,0.584
Content Structure,,0.548


Content Structure,,0.988
,Visual breaks,1.0
,Second person,0.953
,Additional resources,1.0
,Section headings,1.0
Technical Communication,,0.033
,Code inclusion,0.003
,Code explanation,0.0
,Pitfalls,0.097
Total score,,0.511
Content Structure,,0.25


Content Structure,,0.818
,Visual breaks,0.754
,Second person,1.0
,Additional resources,0.566
,Section headings,0.953
Technical Communication,,0.733
,Code inclusion,0.984
,Code explanation,0.668
,Pitfalls,0.547
Total score,,0.776
Content Structure,,0.348


Content Structure,,0.88
,Visual breaks,0.988
,Second person,0.531
,Additional resources,1.0
,Section headings,1.0
Technical Communication,,0.201
,Code inclusion,0.0
,Code explanation,0.0
,Pitfalls,0.602
Total score,,0.54
Content Structure,,0.251


Content Structure,,1.0
,Visual breaks,1.0
,Second person,1.0
,Additional resources,1.0
,Section headings,1.0
Technical Communication,,0.147
,Code inclusion,0.34
,Code explanation,0.002
,Pitfalls,0.098
Total score,,0.573
Content Structure,,0.253


Content Structure,,0.889
,Visual breaks,0.762
,Second person,0.793
,Additional resources,1.0
,Section headings,1.0
Technical Communication,,0.773
,Code inclusion,1.0
,Code explanation,0.934
,Pitfalls,0.385
Total score,,0.831
Content Structure,,0.25


Content Structure,,0.758
,Visual breaks,1.0
,Second person,0.034
,Additional resources,1.0
,Section headings,1.0
Technical Communication,,0.002
,Code inclusion,0.006
,Code explanation,0.0
,Pitfalls,0.0
Total score,,0.38
Content Structure,,0.251


Content Structure,,0.993
,Visual breaks,1.0
,Second person,1.0
,Additional resources,1.0
,Section headings,0.973
Technical Communication,,0.806
,Code inclusion,0.996
,Code explanation,0.664
,Pitfalls,0.758
Total score,,0.9
Content Structure,,0.25


# Fine Tune a Better Blog Post Generator
Now that we've seen that the MachineLearningMastery posts are still significantly better than our prompt generated posts, let's see if we can capture some of that goodness by fine tuning our own model with examples from the original blog. To do so we will:

1. Download a [dataset of previously scraped posts from HuggingFace](https://huggingface.co/datasets/withpi/mlmastery_com_blogs_condensed_mergedhttps://)

2. Filter the dataset to **just the posts that perform really well per our scoring system** (this is the special sauce)

3. Plug that data into our fine tuning SDK endpoint, which will show us a running log as Fine Tuning improves the model's performance on our scoring system

In [9]:
# @title Prepare training data for Fine-tuning by filtering low scoring blogs (<0.7)
from datasets import load_dataset
import pandas

def score(topic:str, blog:str):
  return client.scoring_system.score(
      llm_input=topic,
      llm_output=blog,
      scorer=blog_writer_scorer).total_score

ds = load_dataset("withpi/mlmastery_com_blogs_condensed_merged", split = "train")
ds = ds.map(lambda x: {"score": score(x["topic"], x["blog"])})
ds = ds.filter(lambda x: x["score"] > 0.7)
df = ds.to_pandas()
df

Parameter 'function'=<function <lambda> at 0x7b0a84750220> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/305 [00:00<?, ? examples/s]

Filter:   0%|          | 0/305 [00:00<?, ? examples/s]

,topic,blog,score
0,"The topic of this blog post is: ""Understanding...",# The Da Vinci Code of Data: Mastering The Dat...,0.736165
1,Cross-validation techniques for comprehensive ...,# From Train-Test to Cross-Validation: Advanci...,0.960286
2,Automated Feature Engineering in PyCaret: Stre...,# Automated Feature Engineering in PyCaret\n\n...,0.806478
3,Strategies and techniques for handling imbalan...,# Tips for Handling Imbalanced Data in Machine...,0.836202
4,Finding the optimal feature subset for linear ...,# The Search for the Sweet Spot in a Linear Re...,0.910970
...,...,...,...
132,Visualizing PyTorch Model Architectures Using ...,# Visualizing a PyTorch Model\n\nBy [Adrian Ta...,0.849609
133,Understanding and Working with One-Dimensional...,# One-Dimensional Tensors in Pytorch\n\nBy [Mu...,0.723145
134,Making Predictions with Keras: A Guide to Clas...,# How to Make Predictions with Keras\n\nBy [Ja...,0.814941
135,Visualizing and Interpreting Model Training Me...,# Understand Model Behavior During Training by...,0.926432


In [ ]:
# @title [SLOW - will run for 80+ minutes] Fine tune the model based on the above training data
status = client.training.sft.start_job(
    scorer=blog_writer_scorer,
    examples=[
        {"llm_input": row["topic"], "llm_output": row["blog"]}
        for row in ds
    ],
    base_sft_model="LLAMA_3.1_8B",
    lora_config={"lora_rank": "R_16"},
    system_prompt=system_prompt_for_blog_writer,
    num_train_epochs=10,
)
print(status)

SftStatus(detailed_status=['LAUNCHING'], job_id='sft_jobs:babf2fa45c086088a9e43d648f8ef22e58d7584ce21f17fda4509f2421d84c4c:bf8154b7-77b1-485e-88b5-54d2444b7037', state='QUEUED', trained_models=[])


In [10]:
# @title Monitor the fine-tuning job for completion (watch the Eval_Pi_Score increase!)
from withpi_utils.colab import stream_training_response

SFT_JOB_ID = "sft_jobs:babf2fa45c086088a9e43d648f8ef22e58d7584ce21f17fda4509f2421d84c4c:bf8154b7-77b1-485e-88b5-54d2444b7037"

response = stream_training_response(
    SFT_JOB_ID,
    client.training.sft,
    additional_columns={"Eval_Pi_Score": "contract_score"},
)
if response.state == "ERROR":
  print("The job failed due to:\n{}".format('\n'.join(response.detailed_status[-5:])))
else:
  print("SFT model = {}".format(response.trained_models[0].model_dump_json(indent=2)))

Training Status for sft_jobs:babf2fa45c086088a9e43d648f8ef22e58d7584ce21f17fda4509f2421d84c4c:bf8154b7-77b1-485e-88b5-54d2444b7037


,Step,Epoch,Learning_Rate,Training_Loss,Eval_Loss,Eval_Pi_Score
0,0,0.000000,X,X,1.279658,0.416339
1,3,0.393443,0.00015,1.2192,X,X
2,6,0.786885,0.000194,1.0613,1.199957,0.493448
3,9,1.262295,0.000185,1.45,X,X
4,12,1.655738,0.000176,1.0099,1.147936,0.527657
5,15,2.131148,0.000167,1.3192,X,X
6,18,2.524590,0.000158,1.0675,1.123358,0.598429
7,21,2.918033,0.000148,0.9379,X,X
8,24,3.393443,0.000139,1.2759,1.11116,0.600455
9,27,3.786885,0.00013,0.9087,X,X


SFT model = {
  "contract_score": 0.6664015650749208,
  "epoch": 9.39344262295082,
  "eval_loss": 1.1428908109664917,
  "serving_id": 0,
  "serving_state": "SERVING",
  "step": 66
}


# Test Out & Evaluate Your Fine Tuned Generator

Now our new model is ready to be tested out!

1. First, we'll generate blog posts for the same topics we were looking at before

2. Then we'll score all of these blog posts so we can compare them to the generations by our prompted model

3. Then we'll look at some of the individual examples and their scores Side by Side so we can see how much fine tuning improved our blog posts

In [11]:
# @title Generate blogs using the fine tuned model for evaluation
from datasets import load_dataset
from google.colab import userdata
import time

ds = load_dataset("withpi/mlmastery_com_blogs_condensed_merged", split = "test")
topics = ds["topic"]

# Generate the blogs using fine tuned llama 8B
client.training.sft.load(SFT_JOB_ID)

# Wait for the model to be loaded
while not (client.training.sft.retrieve(SFT_JOB_ID).trained_models[0].serving_state == "SERVING"):
    time.sleep(3)

loop = asyncio.get_running_loop()
new_generated_blogs = await loop.create_task(
    generate_blogs(
        topics,
        model_id="fireworks_ai/0",
        api_base=f"https://api.withpi.ai/v1/training/sft/{SFT_JOB_ID}",
        api_key=os.environ["WITHPI_API_KEY"],
        system_prompt=system_prompt_for_blog_writer
    )
)

Generated a blog for topic# 1: InterviewAce: 365 Data Science's Free AI-Powered Tool for Data Science Interview Preparation
Generated a blog for topic# 4: Evaluating RAG Systems: An Overview of RAGAs and Other Frameworks for Measuring Retrieval Augmented Generation Performance
Generated a blog for topic# 3: A comprehensive guide to Hugging Face's Model Hub and Community platform, including repository management, search functionality, API integration, and community resources for machine learning practitioners.
Generated a blog for topic# 2: Topic: 7 Beginner-Friendly Machine Learning Projects for Hands-On Experience - From Titanic Survival Prediction to Face Detection
Generated a blog for topic# 5: 7-Day Mini-Course on Practical Data Science: From Linear Regression to Random Forests with Real-World Country Data Analysis
Generated a blog for topic# 6: Best practices for efficient machine learning model deployment, including optimization, containerization, CI/CD implementation, performanc

In [12]:
# @title Compare the newly generated blogs against previous ones using the Pi scoring system
from tqdm import tqdm
import pandas as pd

scores = []
generated_scores = []
new_generated_scores = []
for topic, generated_blog, new_generated_blog in tqdm(zip(topics, generated_blogs, new_generated_blogs)):
  generated_score = client.scoring_system.score(
      llm_input=topic,
      llm_output=generated_blog,
      scorer=blog_writer_scorer)
  new_generated_score = client.scoring_system.score(
      llm_input=topic,
      llm_output=new_generated_blog,
      scorer=blog_writer_scorer)
  generated_scores.append(generated_score)
  new_generated_scores.append(new_generated_score)
  score = {'topic': topic, 'generated': generated_score.total_score, 'new generated': new_generated_score.total_score}
  scores.append(score)

df = pd.DataFrame(scores)
df

78it [00:38,  2.04it/s]


,topic,generated,new generated
0,InterviewAce: 365 Data Science's Free AI-Power...,0.222490,0.485366
1,Topic: 7 Beginner-Friendly Machine Learning Pr...,0.397787,0.666840
2,A comprehensive guide to Hugging Face's Model ...,0.297150,0.445552
3,Evaluating RAG Systems: An Overview of RAGAs a...,0.296236,0.469594
4,7-Day Mini-Course on Practical Data Science: F...,0.462908,0.355722
...,...,...,...
73,Packaging and Deploying Python Projects: From ...,0.524812,0.620768
74,Creating and Customizing Dataset Classes in Py...,0.321944,0.715169
75,Implementing Dropout Regularization in PyTorch...,0.473637,0.629395
76,Building and Training a Single Layer Neural Ne...,0.446015,0.766093


In [13]:
# @title Manually inspect new generated blogs against previous ones with scores
from withpi_utils.colab import pretty_print_responses

def pretty_print_blog(i):
  pretty_print_responses(
      response1 = generated_blogs[i].strip("\"").replace("\\n", "\n"),
      response2 = new_generated_blogs[i].strip("\"").replace("\\n", "\n"),
      header="##### Topic: \n" + topics[i].strip("\"").replace("\\n", "\n"),
      left_label="Base (generated)",
      right_label="Test (new generated)",
      scores_left=generated_scores[i],
      scores_right=new_generated_scores[i])

for i in range(10):
  pretty_print_blog(i)
  print("\n\n")

Content Structure,,0.345
,Visual breaks,0.075
,Second person,0.307
,Additional resources,0.0
,Section headings,1.0
Technical Communication,,0.1
,Code inclusion,0.0
,Code explanation,0.0
,Pitfalls,0.299
Total score,,0.222
Content Structure,,0.885


Content Structure,,0.52
,Visual breaks,0.41
,Second person,0.668
,Additional resources,0.0
,Section headings,1.0
Technical Communication,,0.276
,Code inclusion,0.543
,Code explanation,0.285
,Pitfalls,0.0
Total score,,0.398
Content Structure,,0.893


Content Structure,,0.548
,Visual breaks,0.0
,Second person,1.0
,Additional resources,0.193
,Section headings,1.0
Technical Communication,,0.046
,Code inclusion,0.0
,Code explanation,0.0
,Pitfalls,0.138
Total score,,0.297
Content Structure,,0.728


Content Structure,,0.25
,Visual breaks,0.0
,Second person,0.0
,Additional resources,0.0
,Section headings,1.0
Technical Communication,,0.342
,Code inclusion,0.816
,Code explanation,0.211
,Pitfalls,0.0
Total score,,0.296
Content Structure,,0.684


Content Structure,,0.348
,Visual breaks,0.0
,Second person,0.391
,Additional resources,0.0
,Section headings,1.0
Technical Communication,,0.578
,Code inclusion,1.0
,Code explanation,0.734
,Pitfalls,0.0
Total score,,0.463
Content Structure,,0.622


Content Structure,,0.251
,Visual breaks,0.0
,Second person,0.006
,Additional resources,0.0
,Section headings,1.0
Technical Communication,,0.16
,Code inclusion,0.001
,Code explanation,0.0
,Pitfalls,0.479
Total score,,0.206
Content Structure,,0.75


Content Structure,,0.253
,Visual breaks,0.0
,Second person,0.013
,Additional resources,0.0
,Section headings,1.0
Technical Communication,,0.0
,Code inclusion,0.0
,Code explanation,0.0
,Pitfalls,0.0
Total score,,0.127
Content Structure,,0.808


Content Structure,,0.25
,Visual breaks,0.0
,Second person,0.0
,Additional resources,0.0
,Section headings,1.0
Technical Communication,,0.51
,Code inclusion,0.734
,Code explanation,0.346
,Pitfalls,0.449
Total score,,0.38
Content Structure,,0.572


Content Structure,,0.251
,Visual breaks,0.0
,Second person,0.004
,Additional resources,0.0
,Section headings,1.0
Technical Communication,,0.0
,Code inclusion,0.001
,Code explanation,0.0
,Pitfalls,0.0
Total score,,0.126
Content Structure,,0.501


Content Structure,,0.25
,Visual breaks,0.0
,Second person,0.0
,Additional resources,0.0
,Section headings,1.0
Technical Communication,,0.457
,Code inclusion,0.77
,Code explanation,0.479
,Pitfalls,0.122
Total score,,0.353
Content Structure,,0.25
